In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import requests
import os
import time
import json
import math
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import statsmodels.tools.tools as stattools
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#### First task is to create the dataframe of S&P 500 relevant financials and executives. We will constrain analysis to just these 503 companies to narrow the scope of this project. 

In [228]:
#List of S&P 500 tickers from: https://stockmarketmba.com/stocksinthesp500.php
ticker_list = pd.read_csv('SP500_index.csv')['Symbol']
ticker_list

0       AAPL
1       MSFT
2       GOOG
3      GOOGL
4       AMZN
       ...  
498     LUMN
499      ALK
500      NWL
501      VNO
502      TAP
Name: Symbol, Length: 503, dtype: object

In [202]:
# Function to create excel files based on company financials
def income_puller(ticker):
    url=f'https://stockrow.com/api/companies/{ticker}/financials.xlsx?dimension=T&section=Income%20Statement&sort=desc'
    response = requests.get(url)
    with open(os.path.join("StockRow_financials/Excel", f"{ticker}_inc.xlsx"), 'wb') as f:
        f.write(response.content)
    time.sleep(0.25)

# Function to create CSV files based on company files
# Pulling based on ticker
def csv_maker(ticker):
    #this section reads the Excel file and turns into a CSV
    ticker_df = pd.read_excel(f'StockRow_financials/Excel/{ticker}_inc.xlsx').T
    # change the first row to the columns
    ticker_df.columns = ticker_df.iloc[0]
    #remove the first row
    ticker_df = ticker_df.iloc[1:]
    
    ticker_df['Ticker'] = f"{ticker}"
    
    ticker_df.to_csv(f'StockRow_financials/CSV/{ticker}_inc.csv',index_label="Date")

#### This code pulls financial information from StockRow.com and we have left it in as a commented out cell to display the process of excel and CSV creation. 

In [203]:
#This was a one-time request to create the CSVs for each ticker.
#start_time = time.time()
#
#for ticker in ticker_list:
#    try:
#        income_puller(ticker)
#    except:
#        print(f"Ticker {ticker} throws an error")
#    
#print(f"Code took {np.round(time.time() - start_time,2)} seconds to run")

#### This code converts the StockRow workbooks from Excel files to CSV files

In [204]:
start_time = time.time()

for ticker in ticker_list:
    try:
        csv_maker(ticker)
    except:
        print(f"Ticker {ticker} throws an error")
    
print(f"Code took {np.round(time.time() - start_time,2)} seconds to run")

Ticker AAPL throws an error
Ticker MSFT throws an error
Ticker GOOG throws an error
Ticker GOOGL throws an error
Ticker AMZN throws an error
Ticker BRK.B throws an error
Ticker TSLA throws an error
Ticker UNH throws an error
Ticker XOM throws an error
Ticker JNJ throws an error
Ticker WMT throws an error
Ticker V throws an error
Ticker NVDA throws an error
Ticker JPM throws an error
Ticker CVX throws an error
Ticker LLY throws an error
Ticker PG throws an error
Ticker MA throws an error
Ticker HD throws an error
Ticker META throws an error
Ticker BAC throws an error
Ticker PFE throws an error
Ticker ABBV throws an error
Ticker KO throws an error
Ticker MRK throws an error
Ticker PEP throws an error
Ticker COST throws an error
Ticker ORCL throws an error
Ticker TMO throws an error
Ticker AVGO throws an error
Ticker MCD throws an error
Ticker CSCO throws an error
Ticker ACN throws an error
Ticker DHR throws an error
Ticker TMUS throws an error
Ticker ABT throws an error
Ticker WFC throws

Ticker WAT throws an error
Ticker TDY throws an error
Ticker NTRS throws an error
Ticker INVH throws an error
Ticker JBHT throws an error
Ticker CNP throws an error
Ticker AES throws an error
Ticker CHD throws an error
Ticker MOH throws an error
Ticker CLX throws an error
Ticker MAA throws an error
Ticker HOLX throws an error
Ticker WAB throws an error
Ticker DRI throws an error
Ticker EXPD throws an error
Ticker MPWR throws an error
Ticker AMCR throws an error
Ticker J throws an error
Ticker IEX throws an error
Ticker KEY throws an error
Ticker GRMN throws an error
Ticker INCY throws an error
Ticker VTR throws an error
Ticker PKI throws an error
Ticker BALL throws an error
Ticker MOS throws an error
Ticker CMS throws an error
Ticker ABMD throws an error
Ticker BR throws an error
Ticker CAG throws an error
Ticker TTWO throws an error
Ticker SEDG throws an error
Ticker FDS throws an error
Ticker DGX throws an error
Ticker CINF throws an error
Ticker LYV throws an error
Ticker STE throws

#### This code combines all downloaded CSV files (for which we have a ticker) into one Pandas DataFrame

In [229]:
starter_df = pd.read_csv(f'StockRow_financials/CSV/{ticker_list[0]}_inc.csv')

start_time = time.time()

for ticker in ticker_list[1:]:
    try:        
        ticker_df = pd.read_csv(f'StockRow_financials/CSV/{ticker}_inc.csv')
        starter_df = pd.concat([starter_df,ticker_df])
    except:
        print(f'Ticker {ticker} threw an error')
    
        
        
print(f"This code took {time.time() - start_time} seconds")

Ticker META threw an error
Ticker ELV threw an error
Ticker BALL threw an error
Ticker TRGP threw an error
Ticker GEN threw an error
This code took 6.1060028076171875 seconds


In [206]:
#save the DataFrame as one large CSV
starter_df.to_csv('SP500_allFinancials.csv',index=False)

#### Narrow the financial columns, convert "Date" to a DateTime object and create a "Year" field as we ultimately want to analyze the data on a year over year basis.

In [207]:
starter_df = starter_df[['Ticker', 'Date', 'Revenue', 'Gross Profit',
                         'Operating Income', 'Income after Tax', 'Net Income Common']].reset_index(drop=True)

starter_df['Date'] = pd.to_datetime(starter_df['Date'])
starter_df['Year'] = pd.DatetimeIndex(starter_df['Date']).year

In [208]:
#example of grouping the data to find the average financial values for each calendar year
starter_df[starter_df['Ticker']=='AAPL'].groupby('Year').mean()

,Revenue,Gross Profit,Operating Income,Income after Tax,Net Income Common
Year,,,,,
2012,1.646870e+11,6.901900e+10,5.511100e+10,4.174700e+10,4.174700e+10
2013,1.708525e+11,6.525800e+10,5.011225e+10,3.787200e+10,3.787200e+10
2014,1.841935e+11,7.101400e+10,5.336100e+10,4.005850e+10,4.005850e+10
2015,2.263010e+11,9.029050e+10,6.853200e+10,5.141750e+10,5.141750e+10
2016,2.203950e+11,8.646250e+10,6.224650e+10,4.734475e+10,4.734475e+10
2017,2.280935e+11,8.771550e+10,6.122750e+10,4.781425e+10,4.781425e+10
2018,2.574745e+11,9.855900e+10,6.820600e+10,5.710000e+10,5.710000e+10
2019,2.613452e+11,9.912250e+10,6.499925e+10,5.641200e+10,5.641200e+10
2020,2.776220e+11,1.064325e+11,6.831750e+10,5.924500e+10,5.924500e+10


In [209]:
#grouping the data to find average values per year for every ticker
condensed_finances_df = starter_df.groupby(['Ticker','Year']).mean()
condensed_finances_df

Revenue  Gross Profit  Operating Income  Income after Tax  \
Ticker Year                                                                   
A      2013  6.854500e+09  3.569250e+09      9.845000e+08      9.442500e+08   
       2014  5.252250e+09  2.680750e+09      6.885000e+08      4.702500e+08   
       2015  4.619250e+09  2.308250e+09      5.257500e+08      3.070000e+08   
       2016  4.116000e+09  2.130250e+09      5.877500e+08      4.672500e+08   
       2017  4.357750e+09  2.319750e+09      7.500000e+08      6.020000e+08   
...                   ...           ...               ...               ...   
ZTS    2018  5.644000e+09  3.819000e+09      1.673500e+09      1.166000e+09   
       2019  6.092500e+09  4.097500e+09      1.727250e+09      1.430500e+09   
       2020  6.474000e+09  4.500250e+09      1.967500e+09      1.631250e+09   
       2021  7.454000e+09  5.169500e+09      2.354000e+09      1.922500e+09   
       2022  7.964333e+09  5.451667e+09      2.553333e+09      2.074000e+09   

             Net Income Common  
Ticker Year                     
A      2013       9.442500e+08  
       2014       6.605000e+08  
       2015       3.390000e+08  
       2016       4.650000e+08  
       2017       6.020000e+08  
...                        ...  
ZTS    2018       1.171250e+09  
       2019       1.431000e+09  
       2020       1.632250e+09  
       2021       1.925500e+09  
       2022       2.076667e+09  

[5330 rows x 5 columns]

#### We compared the following Apple cell with a manual check of Apple financial data via search engine to ensure the trustworthiness of the method used to group the data.

In [210]:
#sanity check using AAPL
condensed_finances_df.loc['AAPL']

,Revenue,Gross Profit,Operating Income,Income after Tax,Net Income Common
Year,,,,,
2012,1.646870e+11,6.901900e+10,5.511100e+10,4.174700e+10,4.174700e+10
2013,1.708525e+11,6.525800e+10,5.011225e+10,3.787200e+10,3.787200e+10
2014,1.841935e+11,7.101400e+10,5.336100e+10,4.005850e+10,4.005850e+10
2015,2.263010e+11,9.029050e+10,6.853200e+10,5.141750e+10,5.141750e+10
2016,2.203950e+11,8.646250e+10,6.224650e+10,4.734475e+10,4.734475e+10
2017,2.280935e+11,8.771550e+10,6.122750e+10,4.781425e+10,4.781425e+10
2018,2.574745e+11,9.855900e+10,6.820600e+10,5.710000e+10,5.710000e+10
2019,2.613452e+11,9.912250e+10,6.499925e+10,5.641200e+10,5.641200e+10
2020,2.776220e+11,1.064325e+11,6.831750e+10,5.924500e+10,5.924500e+10


#### Now that the financials dataframe is ready, we commence preprocessing for the executive pay dataset that we ultimate want to join with the financials dataset via shared ticker. 

In [230]:
execpay_df = pd.read_csv('main_SCT_pay_file.csv')

execpay_df = execpay_df[execpay_df['ticker'].isin(ticker_list)]

execpay_df

FileNotFoundError: [Errno 2] No such file or directory: 'main_SCT_pay_file.csv'

In [231]:
# Merging the company financials plus the executive pay datasets together to create 
# the main data frame.
main_df = execpay_df.merge(condensed_finances_df,
                           left_on=['ticker','year'], right_on=['Ticker','Year'])

In [232]:
main_df

,id,cik,ticker,name,position,year,salary,bonus,stockAwards,optionAwards,...,total,changeInPensionValueAndDeferredEarnings,CEO,CFO,Interim,Revenue,Gross Profit,Operating Income,Income after Tax,Net Income Common
0,73b3a60ba203743c008330c96b7d8b66,1090872,A,Sam Raha,"Senior Vice President, President Diagnostics a...",2021,563500,0,1541332,0,...,3262887,NaN,False,False,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,983750000.0
1,97393f60cd9f321650e472673daaa70c,1090872,A,Michael R. McMullen,Chief Executive Officer,2021,1280000,0,9165390,0,...,15967631,NaN,True,False,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,983750000.0
2,83b9cc2bca477fe8ce23e0ab56e70c66,1090872,A,Jacob Thaysen,"Senior Vice President, President Life Sciences...",2021,625000,0,1812285,0,...,3802349,NaN,False,False,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,983750000.0
3,7c60bb804071675ae15ec930f6dea190,1090872,A,Robert McMahon,"Senior Vice President, Chief Financial Officer",2021,663500,0,2291271,0,...,4711733,NaN,False,True,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,983750000.0
4,259b710a8befe67c61a2c3dec14f344a,1090872,A,Padraig McDonnell,"Senior Vice President, President Cross-Lab Group",2021,495000,0,1249771,0,...,3243927,NaN,False,False,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,983750000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28435,7b803e9ff0f142d5b1c36b5fdbaced15,1564708,NWS,Gerson Zweifach,General Counsel,2014,1500000,0,0,0,...,1500000,NaN,False,False,False,8.548750e+09,3.453500e+09,4.040000e+08,1.545000e+08,-29750000.0
28436,473bf52703cdbffdc9b311d8dee9101a,1564708,NWS,Bedi Ajay Singh,Chief Financial Officer,2013,655769,713425,496499,0,...,2087442,NaN,False,True,False,8.822667e+09,3.454667e+09,2.525000e+08,2.126667e+08,169000000.0
28437,c36a1d9990a2218f8d2a09b25fb7cd65,1564708,NWS,Robert J. Thomson,Chief Executive Officer,2013,992308,1000000,0,0,...,2661463,NaN,True,False,False,8.822667e+09,3.454667e+09,2.525000e+08,2.126667e+08,169000000.0
28438,7d64a65ecf544ffd0ce4d4ada1607c73,1564708,NWS,Gerson Zweifach,General Counsel,2013,0,0,0,0,...,0,NaN,False,False,False,8.822667e+09,3.454667e+09,2.525000e+08,2.126667e+08,169000000.0


NOTE: The following 10 cells (until the cell that starts with the comment "Start CEO/CFO Classification Model") were set up for another classification purpose to classify a company as growing, stagnating, or shrinking based on executive pay trends. The goal here was, "are changes in executive pay appropriate in line company performance?" We decided to not do this, but we wish to keep the code as it is an interesting project for the future, and it was a large factor in our discussion process of how we wanted to work with this data set to begin with. 

In [214]:
main_df.ticker.value_counts()

A       112
MTCH     99
JCI      92
APA      90
EXC      88
       ... 
CARR     16
CME      14
AMP       5
VTRS      5
OGN       5
Name: ticker, Length: 485, dtype: int64

In [215]:
apple_df = main_df[main_df['ticker']=='AAPL']
apple_df
apple_df_2021 = apple_df[apple_df['year']==2020]
apple_df_2021
rev_2021 = apple_df_2021['Revenue']
rev_2021
# Making sure we can perform calculations with this, looking for 'float' or 'int'
type(rev_2021)

apple_df_2012 = apple_df[apple_df['year']==2012]
rev_2012 = max(apple_df_2012['Revenue'])

subtracted_rev = rev_2021-rev_2012

percent_rev_change = (subtracted_rev/rev_2012)*100

print("Revenue percentage growth over 10 years: ", percent_rev_change)

Revenue percentage growth over 10 years:  117    68.57554
118    68.57554
119    68.57554
120    68.57554
121    68.57554
Name: Revenue, dtype: float64


In [216]:
main_df_ticker_list = main_df['ticker'].unique()
main_df_ticker_list
data_2012 = []
# data_2013 = []
# data_2014 = []
# data_2015 = []
# data_2016 = []
# data_2017 = []
# data_2018 = []
# data_2019 = []
# data_2020 = []
data_2021 = []
data_change_dollars = []
data_change_percentage = []


data = {"Ticker": main_df_ticker_list,
       "Revenue 2012": data_2012,
#         "Revenue 2013": data_2013,
#         "Revenue 2014": data_2014,
#         "Revenue 2015": data_2015,
#         "Revenue 2016": data_2016,
#        "Revenue 2017": data_2017,
#         "Revenue 2018": data_2018,
#         "Revenue 2019": data_2019,
#         "Revenue 2020": data_2020,
        "Revenue 2021": data_2021,
       "Revenue Change $": data_change_dollars,
       "Revenue Change %": data_change_percentage}

for ticker in main_df_ticker_list:
    ticker_df = main_df[main_df["ticker"]==ticker]
    try:
        ticker_df_2012 = ticker_df[ticker_df["year"]==2012]
        rev_2012 = max(ticker_df_2012["Revenue"])
        data_2012.append(rev_2012)
    except:
        data_2012.append("No Data Available")
#     try:
#         ticker_df_2013 = ticker_df[ticker_df["year"]==2013]
#         rev_2013 = max(ticker_df_2013["Revenue"])
#         data_2013.append(rev_2013)
#     except:
#         data_2013.append("No Data Available")
#     try:
#         ticker_df_2014 = ticker_df[ticker_df["year"]==2014]
#         rev_2014 = max(ticker_df_2014["Revenue"])
#         data_2014.append(rev_2014)
#     except:
#         data_2014.append("No Data Available")
#     try:
#         ticker_df_2015 = ticker_df[ticker_df["year"]==2015]
#         rev_2015 = max(ticker_df_2015["Revenue"])
#         data_2015.append(rev_2015)
#     except:
#         data_2015.append("No Data Available")
#     try:
#         ticker_df_2016 = ticker_df[ticker_df["year"]==2016]
#         rev_2016 = max(ticker_df_2016["Revenue"])
#         data_2016.append(rev_2016)
#     except:
#         data_2016.append("No Data Available")
#     try:
#         ticker_df_2017 = ticker_df[ticker_df["year"]==2017]
#         rev_2017 = max(ticker_df_2017["Revenue"])
#         data_2017.append(rev_2017)
#     except:
#         data_2017.append("No Data Available")
#     try:
#         ticker_df_2018 = ticker_df[ticker_df["year"]==2018]
#         rev_2018 = max(ticker_df_2018["Revenue"])
#         data_2018.append(rev_2018)
#     except:
#         data_2018.append("No Data Available")    
#     try:
#         ticker_df_2019 = ticker_df[ticker_df["year"]==2019]
#         rev_2019 = max(ticker_df_2019["Revenue"])
#         data_2019.append(rev_2019)
#     except:
#         data_2019.append("No Data Available")
#     try:
#         ticker_df_2020 = ticker_df[ticker_df["year"]==2020]
#         rev_2020 = max(ticker_df_2020["Revenue"])
#         data_2020.append(rev_2020)
#     except:
#         data_2020.append("No Data Available")
    try:
        ticker_df_2021 = ticker_df[ticker_df['year']==2021]
        rev_2021 = max(ticker_df_2021['Revenue'])
        data_2021.append(rev_2021)
    except:
        data_2021.append("No Data Available")
    try:
        rev_change_dollars = rev_2021 - rev_2012
        data_change_dollars.append(rev_change_dollars)
    except:
        print("No calculation performed")
    try:
        rev_change_percentage = (rev_change_dollars / rev_2012)*100
        data_change_percentage.append(rev_change_percentage)
    except:
        print("No calculation performed")
        
        

new_df = pd.DataFrame(data, index = main_df_ticker_list)
new_df
no_data_2012 = new_df["Revenue 2012"] == "No Data Available" 
no_data_2021 = new_df["Revenue 2021"] == "No Data Available"
no_calc_2012 = new_df.loc[no_data_2012, ["Revenue Change $", "Revenue Change %"]] = "No Calculation Performed"
no_calc_2022 = new_df.loc[no_data_2021, ["Revenue Change $", "Revenue Change %"]] = "No Calculation Performed"
print(new_df.head(15))
print(new_df.shape)



      Ticker         Revenue 2012         Revenue 2021  \
A          A    No Data Available         5952000000.0   
AAPL    AAPL       164687000000.0       354175250000.0   
MSFT    MSFT        72930000000.0       172302750000.0   
AMZN    AMZN        61093000000.0  447553750000.000061   
TSLA    TSLA          413256000.0        44618250000.0   
GOOGL  GOOGL    No Data Available    No Data Available   
GOOG    GOOG        46039000000.0       228366750000.0   
UNH      UNH       110618000000.0       275483250000.0   
JNJ      JNJ        67224000000.0        89659750000.0   
XOM      XOM  480681000000.000122    No Data Available   
JPM      JPM       108074000000.0       127989000000.0   
NVDA    NVDA    No Data Available        20525500000.0   
PG        PG        81646000000.0        76618000000.0   
V          V        10720000000.0        23395750000.0   
CVX      CVX       241909000000.0       129147500000.0   

               Revenue Change $          Revenue Change %  
A      No C

The following code block shows that we have 84 of our 485 companies that do not have data in either 2021 or 2012. Some have both that are missing but overall, we can see that 2012 is the bigger culprit of missing revenue data.

In [217]:
print(f"Missing {new_df['Revenue 2012'].value_counts()['No Data Available']} values in 2012")
print(f"Missing {new_df['Revenue 2021'].value_counts()['No Data Available']} values in 2021")
print(f"Missing {new_df['Revenue Change $'].value_counts()['No Calculation Performed']} change history records overall")

Missing 73 values in 2012
Missing 14 values in 2021
Missing 84 change history records overall


In [243]:
index_no_calc = new_df[new_df['Revenue Change $'] == 'No Calculation Performed'].index
new_df.drop(index_no_calc , inplace=True)
new_df

,Ticker,Revenue 2012,Revenue 2021,Revenue Change $,Revenue Change %,Growth_Rank
AAPL,AAPL,164687000000.0,354175250000.0,189488250000.0,115.059628,128.0
MSFT,MSFT,72930000000.0,172302750000.0,99372750000.0,136.257713,97.0
AMZN,AMZN,61093000000.0,447553750000.000061,386460750000.000061,632.577791,10.0
TSLA,TSLA,413256000.0,44618250000.0,44204994000.0,10696.757942,1.0
GOOG,GOOG,46039000000.0,228366750000.0,182327750000.0,396.02891,25.0
...,...,...,...,...,...,...
ALK,ALK,4657000000.0,4455250000.0,-201750000.0,-4.332188,341.0
DVA,DVA,8186280000.0,11573402000.0,3387122000.0,41.375594,224.0
VNO,VNO,2649217100.0,1524119500.0,-1125097600.0,-42.46906,378.0
DISH,DISH,13181334000.0,17680414750.0,4499080750.0,34.132211,243.0


In [219]:
# Performing a rank on who grew the most and least, by percentage growth not overall growth
new_df['Growth_Rank'] = new_df['Revenue Change %'].rank(ascending=False)
new_df

# Sorting the rank
rslt_df = new_df.sort_values(by = 'Growth_Rank')
# There are three rows with NaN for 2021 Revenue data to drop
rslt_df = rslt_df.dropna()

In [220]:
rslt_df.head(30)

,Ticker,Revenue 2012,Revenue 2021,Revenue Change $,Revenue Change %,Growth_Rank
TSLA,TSLA,413256000.0,44618250000.0,44204994000.0,10696.757942,1.0
FANG,FANG,74962000.0,4948250000.0,4873288000.0,6501.011179,2.0
DXCM,DXCM,99900000.0,2241049950.0,2141149950.0,2143.293243,3.0
NOW,NOW,243712000.0,5357500000.0,5113788000.0,2098.291426,4.0
CNC,CNC,8110000000.0,120285750000.0,112175750000.0,1383.178175,5.0
INCY,INCY,297059000.0,2830670425.0,2533611425.0,852.898389,6.0
REGN,REGN,1378477100.0,12799075000.0,11420597900.0,828.493843,7.0
NFLX,NFLX,3609281800.0,28076899750.0,24467617950.0,677.908218,8.0
EPAM,EPAM,433799000.0,3239886500.0,2806087500.0,646.863524,9.0
AMZN,AMZN,61093000000.0,447553750000.000061,386460750000.000061,632.577791,10.0


In [221]:
rslt_df[rslt_df['Revenue Change %']< 0].count()

Ticker              70
Revenue 2012        70
Revenue 2021        70
Revenue Change $    70
Revenue Change %    70
Growth_Rank         70
dtype: int64

In [222]:
print(f"Median of Percent Revenue Change is {rslt_df['Revenue Change %'].median()}")
print(f"Mean of Percent Revenue Change is {rslt_df['Revenue Change %'].mean()}")

Median of Percent Revenue Change is 52.98032445273718
Mean of Percent Revenue Change is 148.36562357985858


In [223]:
rslt_df['Label'] = ['Growing Company' if x >= 0
                    else 'Declining Company' 
                    for x in rslt_df['Revenue Change %']]
rslt_df['Label'].value_counts()

Growing Company      328
Declining Company     70
Name: Label, dtype: int64

#### Below is the first cell of the classification model predicting CEO or CFO.

In [233]:
# Start CEO/CFO Classification Model
main_df.head(15)
main_df = main_df.drop_duplicates()
main_df.loc[main_df['ticker']=='A']

,id,cik,ticker,name,position,year,salary,bonus,stockAwards,optionAwards,...,total,changeInPensionValueAndDeferredEarnings,CEO,CFO,Interim,Revenue,Gross Profit,Operating Income,Income after Tax,Net Income Common
0,73b3a60ba203743c008330c96b7d8b66,1090872,A,Sam Raha,"Senior Vice President, President Diagnostics a...",2021,563500,0,1541332,0,...,3262887,NaN,False,False,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,9.837500e+08
1,97393f60cd9f321650e472673daaa70c,1090872,A,Michael R. McMullen,Chief Executive Officer,2021,1280000,0,9165390,0,...,15967631,NaN,True,False,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,9.837500e+08
2,83b9cc2bca477fe8ce23e0ab56e70c66,1090872,A,Jacob Thaysen,"Senior Vice President, President Life Sciences...",2021,625000,0,1812285,0,...,3802349,NaN,False,False,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,9.837500e+08
3,7c60bb804071675ae15ec930f6dea190,1090872,A,Robert McMahon,"Senior Vice President, Chief Financial Officer",2021,663500,0,2291271,0,...,4711733,NaN,False,True,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,9.837500e+08
4,259b710a8befe67c61a2c3dec14f344a,1090872,A,Padraig McDonnell,"Senior Vice President, President Cross-Lab Group",2021,495000,0,1249771,0,...,3243927,NaN,False,False,False,5.952000e+09,3.193250e+09,1.175500e+09,9.837500e+08,9.837500e+08
10,6a9fd7caafd5ff9409e40f8c289e1cd3,1090872,A,Jacob Thaysen,"Senior Vice President, President Life Sciences...",2020,614904,0,2117779,0,...,3074102,NaN,False,False,False,5.258500e+09,2.813250e+09,8.352500e+08,7.142500e+08,7.142500e+08
11,ccef820f858971ef639e1266f4b96840,1090872,A,Michael R. McMullen,Chief Executive Officer,2020,1172853,0,11190749,0,...,13610481,NaN,True,False,False,5.258500e+09,2.813250e+09,8.352500e+08,7.142500e+08,7.142500e+08
12,0afb57350eef3b644a4f9802490ce25f,1090872,A,Robert McMahon,"Senior Vice President, Chief Financial Officer",2020,638333,0,2614535,0,...,3665071,NaN,False,True,False,5.258500e+09,2.813250e+09,8.352500e+08,7.142500e+08,7.142500e+08
13,5c5621b65ca11ff037e274c80b191cd2,1090872,A,Sam Raha,"Senior Vice President, President Diagnostics and",2020,538782,0,1673282,0,...,2471060,NaN,False,False,False,5.258500e+09,2.813250e+09,8.352500e+08,7.142500e+08,7.142500e+08
14,4deeb4d5b673e6f653a86a1eddda5990,1090872,A,Genomics Group,"Michael Tang Senior Vice President, General Co...",2020,561026,0,1568688,0,...,2439705,NaN,False,False,False,5.258500e+09,2.813250e+09,8.352500e+08,7.142500e+08,7.142500e+08


In [242]:
# Need to drop duplicate Agilent ones with the name "Group" or "human resources"
main_df = main_df.drop(main_df[(main_df.name == 'Group') | 
                          (main_df.name == 'Human Resources')].index)
# Creating the separate dataframes for CEO/CFO so that we can perform aggregate 
# functions separately.
df_CEO = main_df[(main_df['CEO'] == True)] 
df_CFO = main_df[(main_df['CFO'] == True)] 

# Getting sum of values per ticker.
df_CEO = df_CEO.groupby('ticker').sum()
df_CFO = df_CFO.groupby('ticker').sum()

# Subsetting for the fields we want
df_CEO = df_CEO[['CEO','salary', 'bonus', 'stockAwards', 
                 'optionAwards','nonEquityIncentiveCompensation', 
                 'total', 'Revenue', 'Net Income Common']]
df_CEO.loc[df_CEO["CEO"] > 0, "CEO"] = True

df_CFO = df_CFO[['CEO','salary', 'bonus', 'stockAwards', 
                 'optionAwards','nonEquityIncentiveCompensation', 
                 'total', 'Revenue', 'Net Income Common']]
df_CFO.loc[df_CFO["CEO"] == 0, "CEO"] = False

In [226]:
# Preparing to join the data frames back together
frames = [df_CEO, df_CFO]
  
# Joining the two data frames together.
new_main = pd.concat(frames)
new_main = new_main.sort_values(by=['ticker'])


# NOTE: The following commented out code gets percentages of executive pay as part of revenue.
# It is commented out because it is not used in the final classification model but is 
# interesting as a potential variable in future projects with this dataset. 
# new_main['salary/revenue'] = new_main['salary'] / new_main['Revenue']
# new_main['bonus/revenue'] = new_main['bonus'] / new_main['Revenue']
# new_main['stockAwards/revenue'] = new_main['stockAwards'] / new_main['Revenue']
# new_main['optionAwards/revenue'] = new_main['optionAwards'] / new_main['Revenue']
# new_main['nonEquityIncentiveCompensation/revenue'] = new_main['nonEquityIncentiveCompensation'] / new_main['Revenue']
# new_main['all_Equity_Awards'] = new_main['stockAwards'] + new_main['optionAwards']

## Dropping Where CEO is not equal to True or False
indexNumbers = new_main[ (new_main['CEO'] == 3) | 
                 (new_main['CEO'] == 2) | (new_main['CEO'] == 5)].index
new_main.drop(indexNumbers , inplace=True)

## Converting True/False CEO column from object to boolean because the model
## was not recognizing the y-predictor type.
new_main['CEO'] = new_main['CEO'].astype(bool)
print(new_main.dtypes)
new_main

CEO                                          bool
salary                                      int64
bonus                                       int64
stockAwards                                 int64
optionAwards                                int64
nonEquityIncentiveCompensation              int64
total                                       int64
Revenue                                   float64
Net Income Common                         float64
salary/revenue                            float64
bonus/revenue                             float64
stockAwards/revenue                       float64
optionAwards/revenue                      float64
nonEquityIncentiveCompensation/revenue    float64
all_Equity_Awards                           int64
dtype: object


,CEO,salary,bonus,stockAwards,optionAwards,nonEquityIncentiveCompensation,total,Revenue,Net Income Common,salary/revenue,bonus/revenue,stockAwards/revenue,optionAwards/revenue,nonEquityIncentiveCompensation/revenue,all_Equity_Awards
ticker,,,,,,,,,,,,,,,
A,True,11458269,0,68552878,15107596,16133895,115188782,5.834650e+10,7.675250e+09,0.000196,0.000000,0.001175,0.000259,0.000277,83660474
A,False,5085129,3000000,19646031,2999887,4580051,36893668,4.623975e+10,6.070500e+09,0.000110,0.000065,0.000425,0.000065,0.000099,22645918
AAL,True,2441140,0,130227654,0,10203331,159413533,4.842888e+11,1.304125e+10,0.000005,0.000000,0.000269,0.000000,0.000021,130227654
AAL,False,7218184,0,38471191,0,11200278,62707472,4.588178e+11,1.355525e+10,0.000016,0.000000,0.000084,0.000000,0.000024,38471191
AAP,True,9668318,3485000,44487383,14649081,8633582,83061159,9.050150e+10,4.663842e+09,0.000107,0.000039,0.000492,0.000162,0.000095,59136464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,True,12679100,0,59775901,12967242,19148913,105531711,4.446959e+10,3.304858e+09,0.000285,0.000000,0.001344,0.000292,0.000431,72743143
ZION,False,6419692,3652500,5839982,985436,1808345,17642731,2.865551e+10,5.211822e+09,0.000224,0.000127,0.000204,0.000034,0.000063,6825418
ZION,True,21152988,11702000,15629354,3350878,7071579,58692242,7.047798e+10,1.245729e+10,0.000300,0.000166,0.000222,0.000048,0.000100,18980232


#### We have now finished preprocessing for our data to run through classification models

In [227]:
new_main.to_csv('new_main_df.csv',index=False)